In [1]:
# Import necessary libraries
import ipywidgets as widgets
from IPython.display import display, HTML
# from LazyQML.lazyqml.lazyqml import QuantumClassifier
import pandas as pd
from sklearn.model_selection import train_test_split

import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '../lazyqml/')

from Global.globalEnums import *
from lazyqml import QuantumClassifier
import itertools

# Add custom CSS for better aesthetics
display(HTML("""
<style>
    .widget-label {
        font-weight: bold;
        font-size: 14px;
    }
    .slider {
        width: 500px !important;
    }
    .widget-box {
        background-color: #f9f9f9;
        padding: 15px;
        border-radius: 10px;
        box-shadow: 2px 2px 10px rgba(0, 0, 0, 0.1);
        margin-bottom: 15px;
    }
</style>
"""))

In [2]:

# Widgets for user input with enhanced styles and multi-selection
epochs_widget = widgets.IntText(
    min=1, 
    max=1000, 
    step=1, 
    value=10, 
    description="Epochs:",
    style={'description_width': '150px'}, 
    layout=widgets.Layout(width='500px'))

layers_widget = widgets.IntText(
    value=7.5,
    min=0,
    max=100,
    step=1,
    description='Layers:',
    disabled=False,
    style={'description_width': '150px'},
    layout=widgets.Layout(width='500px')
)

nqubits_widget = widgets.IntText(
    min=1,
    max=30, 
    step=1, 
    value=4, 
    description="Qubits:",
    style={'description_width': '150px'}, 
    layout=widgets.Layout(width='500px')
)

randomstate_widget = widgets.IntText(
    value=1234, 
    description="Seed:",
    style={'description_width': '150px'}, 
    layout=widgets.Layout(width='300px')
)

runs_widget = widgets.IntText(
    min=1,
    max=10, 
    step=1, 
    value=1, 
    description="Runs:",
    style={'description_width': '150px'}, 
    layout=widgets.Layout(width='300px')
)

# # Classifier multi-selection using a Dropdown with multiple selection enabled
# classifiers_options = ['qsvm', 'qnn', 'qnnbag']
# classifiers_widget = widgets.SelectMultiple(
#     options=classifiers_options, value=['qsvm'], description='Classifiers:',
#     style={'description_width': '150px'}, layout=widgets.Layout(width='500px')
# )

# Models
qsvm_checkbox = widgets.Checkbox(
    value=False,
    description='QSVM',
    disabled=False,
    style={'description_width': '150px'}, 
    layout=widgets.Layout(width='300px')
)

qnn_checkbox = widgets.Checkbox(
    value=False,
    description='QNN',
    disabled=False,
    style={'description_width': '150px'}, 
    layout=widgets.Layout(width='300px')
)

qnn_bag_checkbox = widgets.Checkbox(
    value=False,
    description='QNN Bagging',
    disabled=False,
    style={'description_width': '150px'}, 
    layout=widgets.Layout(width='300px')
)

# Ansatzs
hp_checkbox = widgets.Checkbox(
    value=False,
    description='HCzRx',
    disabled=False,
    style={'description_width': '150px'}, 
    layout=widgets.Layout(width='300px')
)

tt_checkbox = widgets.Checkbox(
    value=False,
    description='Tree Tensor',
    disabled=False,
    style={'description_width': '150px'}, 
    layout=widgets.Layout(width='300px')
)

two_checkbox = widgets.Checkbox(
    value=False,
    description='Two Local',
    disabled=False,
    style={'description_width': '150px'}, 
    layout=widgets.Layout(width='300px')
)

hwe_checkbox = widgets.Checkbox(
    value=False,
    description='Hardware Efficient',
    disabled=False,
    style={'description_width': '150px'}, 
    layout=widgets.Layout(width='300px')
)

# Embeddings
rx_checkbox = widgets.Checkbox(
    value=False,
    description='Rotation X',
    disabled=False,
    style={'description_width': '150px'}, 
    layout=widgets.Layout(width='300px')
)

ry_checkbox = widgets.Checkbox(
    value=False,
    description='Rotation Y',
    disabled=False,
    style={'description_width': '150px'}, 
    layout=widgets.Layout(width='300px')
)

rz_checkbox = widgets.Checkbox(
    value=False,
    description='Rotation Z',
    disabled=False,
    style={'description_width': '150px'}, 
    layout=widgets.Layout(width='300px')
)

zz_checkbox = widgets.Checkbox(
    value=False,
    description='ZZ',
    disabled=False,
    style={'description_width': '150px'}, 
    layout=widgets.Layout(width='300px')
)

amp_checkbox = widgets.Checkbox(
    value=False,
    description='Amplitude',
    disabled=False,
    style={'description_width': '150px'}, 
    layout=widgets.Layout(width='300px')
)



# Ansatz multi-selection with Dropdown widget for intuitive multiple selection
# ansatzs_options = ['HPzRx', 'tree_tensor', 'two_local', 'hardware_efficient']
# ansatzs_widget = widgets.SelectMultiple(
#     options=ansatzs_options, value=['HPzRx'], description='Ansatzs:',
#     style={'description_width': '150px'}, layout=widgets.Layout(width='500px'))

# # Embeddings multi-selection using Dropdown
# embeddings_options = ['amplitude_embedding', 'ZZ_embedding', 'rx_embedding', 'rz_embedding', 'ry_embedding']
# embeddings_widget = widgets.SelectMultiple(
#     options=embeddings_options, value=['amplitude_embedding'], description='Embeddings:',
#     style={'description_width': '150px'}, layout=widgets.Layout(width='500px'))

# Verbose switch
verbose_widget = widgets.Checkbox(
    value=False, description="Verbose",
    style={'description_width': '150px'}, layout=widgets.Layout(width='500px'))

# Number of samples as a percentage (from 0.0 to 1.0)
nsamples_widget = widgets.FloatSlider(
    value=1.0,
    min=0.0, 
    max=1.0, 
    step=0.01, 
    description="Samples - [0, 1]:",
    style={'description_width': '150px'}, 
    layout=widgets.Layout(width='100px'))


# Number of estimators (for QNN Bagging)
nestimators_widget = widgets.IntText(
    min=1, max=50, step=1, value=10, description="Estimators:",
    style={'description_width': '150px'}, layout=widgets.Layout(width='500px'))

# Function to display widgets in a nice box layout (stack widgets vertically)
def display_widgets():
    widget_box = widgets.VBox([
        widgets.HTML("<h3>Quantum Classifier Configuration</h3>"),
        widgets.VBox([widgets.Label("General model options:"), epochs_widget, layers_widget, nqubits_widget]),
        widgets.VBox([widgets.Label("Classifiers:"), qsvm_checkbox, qnn_checkbox, qnn_bag_checkbox]),
        widgets.VBox([widgets.Label("Ansatzs:"), hp_checkbox, tt_checkbox, two_checkbox, hwe_checkbox]),
        widgets.VBox([widgets.Label("Embeddings:"), rx_checkbox, ry_checkbox, rz_checkbox, zz_checkbox, amp_checkbox]),
        widgets.VBox([widgets.Label("Bagging options:"), nestimators_widget, nsamples_widget]),
        widgets.VBox([widgets.Label("Other options:"), runs_widget, randomstate_widget, verbose_widget])
    ], layout=widgets.Layout(border='solid 1px #cccccc', padding='20px', border_radius='10px'))
    
    display(widget_box)


# Get options functions
classifier_widgets = [qnn_checkbox, qnn_bag_checkbox, qsvm_checkbox]
ansatz_widgets = [hp_checkbox, tt_checkbox, two_checkbox, hwe_checkbox]
emb_widgets = [rx_checkbox, rz_checkbox, ry_checkbox, zz_checkbox, amp_checkbox]

classifier_enums = Model.list()
classifier_enums.remove(Model.ALL)

ansatz_enums = Ansatzs.list()
ansatz_enums.remove(Ansatzs.ALL)

emb_enums = Embedding.list()
emb_enums.remove(Embedding.ALL)

get_vals = lambda x: x.value

def get_classifiers():
    return itertools.compress(classifier_enums, map(get_vals, classifier_widgets))

def get_ansatzs():
    return itertools.compress(ansatz_enums, map(get_vals, ansatz_widgets))

def get_emb():
    return itertools.compress(emb_enums, map(get_vals, emb_widgets))

# Display
display_widgets()


In [3]:
# Sample DataFrame input (can be customized by the user)
upload_button = widgets.FileUpload(accept=".csv", multiple=False)

def load_dataframe(change):
    if len(upload_button.value) > 0:
        # Read the uploaded CSV file content
        content = next(iter(upload_button.value.values()))['content']
        data = pd.read_csv(pd.compat.BytesIO(content))
        display(data.head())  # Show the first few rows of the dataframe
        return data
    else:
        return None

upload_button.observe(load_dataframe, names='value')
display(upload_button)

FileUpload(value=(), accept='.csv', description='Upload')

In [4]:
# Create an output widget to capture the print statements
output_display = widgets.Output()

# Function to train the model and display outputs
def train_model(data=None):
    try:
        classifiers = set(get_classifiers())
        ansatzs = set(get_ansatzs())
        emb = set(get_emb())

        print(f"Initializing QuantumClassifier with:\nVerbose: {verbose_widget.value}\nRandom State: {randomstate_widget.value}\nNumber of Qubits: {nqubits_widget.value}\nNumber of Layers: {layers_widget.value}\nClassifiers: {classifiers}\nEmbeddings: {emb}\nAnsatzs: {ansatzs}\nEpochs: {epochs_widget.value}\nSamples: {nsamples_widget.value}\nEstimators: {nestimators_widget.value}")
        # Split the dataset into train and test
        X = data.data  # Assuming the target column is named 'target'
        y = data.target
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=randomstate_widget.value)

        # Initialize QuantumClassifier with selected parameters
        clf = QuantumClassifier(
            verbose=verbose_widget.value,
            randomSate=randomstate_widget.value,
            nqubits=nqubits_widget.value,
            numLayers=layers_widget.value,
            classifiers=classifiers,
            ansatzs=ansatzs,
            embeddings=emb,
            epochs=epochs_widget.value,
            maxSamples=nsamples_widget.value,
            numPredictors=nestimators_widget.value,
            runs=runs_widget.value
        )

        # Call the fit method and print the models and results
        clf.fit(X_train=X_train,y_train=y_train,X_test=X_test,y_test=y_test)
        # display(models)  # Display the resulting models
    except Exception as e:
        # print(e)
        print("No data provided. Please upload a dataset.")

# Train button to trigger the training process
train_button = widgets.Button(description="Train Model", button_style='success')

def on_train_button_click(b):
    # Assuming data is loaded using the 'upload_button' widget
    data = load_dataframe(None)  # Call to load the dataframe
    train_model(data)

train_button.on_click(on_train_button_click)
display(train_button)


SyntaxError: invalid syntax (3568351874.py, line 30)